In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
n_px = 512
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor
from torchvision.transforms import InterpolationMode
BICUBIC = InterpolationMode.BICUBIC

def _convert_image_to_rgb(image):
    return image.convert("RGB")

preprocess = Compose([
        Resize(n_px, interpolation=BICUBIC),
        CenterCrop(n_px),
        _convert_image_to_rgb,
        ToTensor()
    ])

In [ ]:
from lidar_clippin.loader import build_loader, OnceImageLidarDataset

dataset = OnceImageLidarDataset("/proj/nlp4adas/datasets/once", preprocess, split='val')

In [ ]:
nice_frames = []

for i, (image, lidar) in enumerate(dataset):
    plt.figure(dpi=200)
    plt.imshow(rearrange(image, "c h w -> h w c"))
    plt.show()
    if i == 5:
        break


In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import matplotlib.pyplot as plt
from einops import rearrange
import random

In [ ]:
# SAVED = []
# SAVED = SAVED_BACKUP
INDEX = 0

In [ ]:
button_next = widgets.Button(description='Next')
button_save = widgets.Button(description='Save')

out = widgets.Output()
def on_next_clicked(_):
    with out:
        clear_output()
        idx = globals()['INDEX']
        plt.figure(dpi=200)
        plt.imshow(rearrange(dataset[idx][0], "c h w -> h w c"))
        plt.title(str(idx))
        plt.axis('off')
        plt.show()
        globals()['INDEX'] = random.randint(0,len(dataset))

def on_save_clicked(_):
    print(globals()['INDEX'])
    globals()['SAVED'].append(globals()['INDEX'])
        
# linking button and function together using a button's method
button_next.on_click(on_next_clicked)
button_save.on_click(on_save_clicked)


In [ ]:
# displaying button and its output together
widgets.VBox([button_next, button_save, out])

In [ ]:
SAVED.append(41819)

In [ ]:
print(SAVED)

In [ ]:
from matplotlib import gridspec
nrow, ncol = 9, 6
plt.figure(figsize=(ncol+1, nrow+1), dpi=400)
gs = gridspec.GridSpec(nrow, ncol,
         wspace=0.0, hspace=0.0, 
         top=1.-0.5/(nrow+1), bottom=0.5/(nrow+1), 
         left=0.5/(ncol+1), right=1-0.5/(ncol+1)) 

for i in range(nrow):
    for j in range(ncol):        
        saved_idx = i*ncol+j
        if saved_idx >= len(SAVED):
            break
        saved = SAVED[saved_idx]
        ax = plt.subplot(gs[i,j])
        ax.imshow(rearrange(dataset[saved][0], "c h w -> h w c"))
        ax.set_xticklabels([])
        ax.set_yticklabels([])
plt.show()


In [ ]:
means = torch.tensor([0.48145466, 0.4578275, 0.40821073], device="cpu")
stds = torch.tensor([0.26862954, 0.26130258, 0.27577711], device="cpu")

from matplotlib import gridspec
nrow, ncol = 9, 6
plt.figure(figsize=(ncol+1, nrow+1), dpi=400)
gs = gridspec.GridSpec(nrow, ncol,
         wspace=0.0, hspace=0.0, 
         top=1.-0.5/(nrow+1), bottom=0.5/(nrow+1), 
         left=0.5/(ncol+1), right=1-0.5/(ncol+1)) 

for i in range(nrow):
    for j in range(ncol):        
        saved_idx = i*ncol+j
        if saved_idx >= len(SAVED):
            break
        saved = SAVED[saved_idx]
        ax = plt.subplot(gs[i,j])
        ax.imshow(rearrange(dataset_clip[saved][0], "c h w -> h w c")*stds+means)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
plt.show()


## Generate lidar features for selected frames

In [ ]:
# Uncomment to use old search
# SAVED = [38, 8668, 26582, 44699, 30525, 26905, 36073, 15578, 23522, 26427, 670, 46502, 38413, 6106, 37222, 43701, 29328, 44228, 39683, 24982, 26959, 18451, 34644, 14102, 35729, 18588, 26000, 38733, 31243, 7656, 4382, 33159, 36259, 1572, 27598, 38137, 7666, 43209, 9928, 2794, 1966, 5309, 42876]

In [ ]:
from lidar_clippin.model.sst import build_sst
from mmcv.runner import load_checkpoint
from lidar_clippin.model.sst import LidarEncoderSST
import clip
import os
from train import LidarClippin

clip_model, clip_preprocess = clip.load("ViT-B/32")
lidar_encoder = LidarEncoderSST("model/sst_encoder_only_config.py")
model = LidarClippin(lidar_encoder, clip_model, 1, 1)
checkpoint_name = "35vsmuyp/epoch=97-step=32842.ckpt"
load_checkpoint(model, os.path.join("/proj/nlp4adas/checkpoints/", checkpoint_name), map_location="cpu")
model.to(device)
pass

In [ ]:
from lidar_clippin.loader import build_loader, OnceImageLidarDataset

lidars, images = [], []

dataset_clip = OnceImageLidarDataset("/proj/nlp4adas/datasets/once", clip_preprocess, split='val')
for saved in SAVED:
    img, lid = dataset_clip[saved]
    lidars.append(lid.to(device))
    images.append(img.to(device))    

In [ ]:
feature, _ = model.lidar_encoder(lidars, return_attention=False)
torch.save(feature, 'lidar_out.pt')

In [ ]:
# conceptual - no beam
CAPTIONS_conc = ['truck on the highway in city.', 'a car driving on a road.', 'driving in the rain on a cloudy day.', 'a car driving through a city.', 'a pedestrian crossing in the city.', 'driving in the rain on a cloudy day.', 'time lapse of traffic in the city.', 'driving through a tunnel in the city.', 'automobile model in the city.', 'driving on a highway in the rain.', 'time lapse of a busy street.', 'driving in the rain on a road.', 'driving in the evening, time lapse.', 'a view of a street with trees and bushes.', 'time lapse of traffic on a busy street.', 'driving in the rain on a city street.', 'a man rides his bike through the streets.', 'a car driving through a city.', 'driving in the evening, driving in the evening.', 'driving in the rain on a highway.', 'driving in the rain on a cloudy day.', 'driving on a highway in the rain.', 'driving in the city at night.', 'driving in the rain on a cloudy day.', 'the car was parked in front of the house.', 'driving in a car on a highway.', 'automobile model on the street.', 'driving in the evening on a busy street.', 'a man walks down a street in the rain.', 'a man walks down a street.', 'driving in the city on a sunny day.', 'driving in the rain on a city street.', 'driving in the city at night.', 'a man rides his bike through the city.', 'driving on a road in the rain.', 'driving in the city, cars and people.', 'driving on a highway in the city.', 'driving through the city streets.', 'a man walks down a path in the park.', 'driving in the city, cars and traffic.', 'a car driving on a road.', 'bus stop in the city.', 'traffic jam on the highway.']
# # conceptual - beam
CAPTIONS_conc_beam = ['traffic jam on the highway.', 'cars driving on a road in the city.', 'driving in the rain on a cloudy day.', 'driving in the rain on a cloudy day.', 'pedestrian walkway in the city.', 'driving in the rain on a cloudy day.', 'time lapse of a busy street.', 'driving through a tunnel in the city.', 'automobile model on the street.', 'driving through the city on a rainy day.', 'time lapse of a traffic jam on a highway.', 'car driving on a road in the rain.', 'time lapse of a busy street at night.', 'driving along a road in the city.', 'time lapse of traffic in the city.', 'driving in heavy fog on the highway.', '4k pov driving through the busy city streets.', 'cars driving on a road in the rain.', 'driving on a highway at night.', 'rainy day in the city.', 'driving in the rain on a cloudy day.', 'driving through the city on a rainy day.', 'time lapse of traffic in the city.', 'driving in the rain on a cloudy day.', 'automobile model parked in front of a house.', 'car driving through the city.', 'automobile model on the street.', 'driving in the city at night.', '4k pov driving through the busy city streets.', '4k pov driving through the busy city streets.', 'driving in the city on a sunny day.', 'pov driving through the city on a rainy day.', 'driving a car in the city.', 'a man rides his scooter down a busy street.', 'driving on a road in the rain.', 'traffic jam in the city.', 'traffic jam on the highway.', 'driving along a road in the city.', 'driving through the park on a sunny day.', 'car driving on the road.', 'car driving on the road.', 'bus stop in the city.', 'traffic jam and gridlock in the city.']
# # coco - no beam
CAPTIONS_coco = ['A large truck is driving down the street.', 'A street with a fire hydrant and trees.', 'A street with a traffic light and buildings in the background.', 'A street with a bunch of trees and a fire hydrant.', 'A street with a fence and trees and a skateboarder.', 'A street with a traffic light and a bridge in the background.', 'A street with a traffic light and trees.', 'A view of a street with a bridge in the background.', 'A street with a stop sign and a green traffic light.', 'A city street with a traffic light and a bus.', 'A street with a traffic light and buildings in the background.', 'A car is driving down a street with trees.', 'A street with a stop sign and a fire hydrant.', 'A view of a tree lined street with a stop sign in the foreground.', 'A white car is driving down a street.', 'A car is driving down a street with fog.', 'A street with a stop sign and trees.', 'A street with a traffic light and a bus on the road.', 'A car is driving down a street with fog.', 'A car driving down a street under a traffic light.', 'A city street with a red traffic light.', 'A city street with cars and traffic lights.', 'A city street filled with lots of traffic.', 'A street with a stop light and trees.', 'A car driving down a street next to a traffic light.', 'A city street filled with lots of traffic.', 'A city street with a traffic light and trees.', 'A street with a lot of traffic and a lot of trees.', 'A car is stopped at a red light.', 'A street with a fire hydrant and trees.', 'A city street filled with lots of traffic.', 'A car driving down a street next to a traffic light.', 'A car stopped at a red light on a busy street.', 'A street with a couple of motorcycles and a van.', 'A street with a fence and trees and a traffic light.', 'A city street with cars and street lights.', 'A car is stopped at a red light.', 'A street with a tree and a stop sign.', 'A street with a tree and a fire hydrant.', 'A car is stopped at a red light.', 'A car driving down a street next to a traffic light.', 'A bus is parked on the side of the road.', 'A busy street with cars and a red stoplight.', 'A street with a traffic light and street signs.', 'A car and a motorcycle are stopped at a red light.', 'A bus is driving down a busy street.', 'A bus driving down a street next to a tall building.', 'A busy street with cars and trucks on it.', 'A woman riding a skateboard down a street.', 'A white car parked in front of a building.', 'A view of a park bench from a distance.', 'A view of a street with a traffic light.', 'A city street with cars driving down it.', 'A street with a traffic light and cars on it.', 'A busy street with cars and traffic lights.']
# CAPTIONS_coco = ['A large truck is driving down the road.', 'A street with a fire hydrant and trees.', 'A city street with cars and a traffic light.', 'A street with a bunch of traffic lights and trees.', 'A city street with a bridge and trees.', 'A street with a stop sign and trees.', 'A city street with cars and a traffic light.', 'A view of a street with a bridge in the background.', 'A silver car parked in front of a tall building.', 'A red stop sign sitting on the side of a road.', 'A street with a traffic light and buildings in the background.', 'A car is driving down a street with a stop sign.', 'A city street at night with a traffic light.', 'A view of a tree lined street with a stop sign in the foreground.', 'A city street with traffic lights and street signs.', 'A car is driving down a street with fog.', 'A street with a traffic light and buildings in the background.', 'A street with a traffic light and a bus on the road.', 'A car is driving down a street at night.', 'A bus driving down a street next to a tall building.', 'A view of a street with a stop sign and trees.', 'A bus driving down a street next to a bridge.', 'A busy city street with cars and traffic lights.', 'A street with a stop light and trees.', 'A car parked in front of a building next to a street sign.', 'A car is stopped at a red light.', 'A white car is driving down a street.', 'A street with cars and street lights at night.', 'A view of a street with a stop sign and buildings.', 'A street with a fire hydrant and trees.', 'A city street filled with lots of traffic.', 'A car is driving down a street with fog.', 'A city street filled with lots of traffic.', 'A man riding a motorcycle down a street next to tall buildings.', 'A street with a traffic light and trees.', 'A busy street with cars and a lot of traffic.', 'A car is stopped at a red light.', 'A street with a tree and a stop sign.', 'A street with a tree and a fire hydrant.', 'A car is stopped at a red light.', 'A car driving down a street next to a traffic light.', 'A bus is parked on the side of the road.', 'A bunch of cars that are sitting in the street.']
# # coco - beam
CAPTIONS_coco_beam = ['A large semi truck on a city street.', 'A view of a street with a fire hydrant on the side of the road.', 'A couple of red traffic lights sitting on the side of a road.', 'A couple of red fire hydrants sitting on the side of a road.', 'A couple of skateboards that are in the street.', 'A view of a street in the middle of winter.', 'A city street filled with lots of traffic.', 'A view of a street with a bridge in the background.', 'A green stop sign sitting on the side of a road.', 'A city street filled with lots of traffic.', 'A traffic light sitting on the side of a road.', 'A couple of cars driving down a street next to trees.', 'A traffic light sitting on the side of a road.', 'A city street lined with trees and bushes.', 'A white car driving down a street next to tall buildings.', 'A traffic light sitting on the side of a road.', 'A view of a street with a stop sign in the middle of it.', 'A city street filled with lots of traffic.', 'A city street filled with lots of traffic.', 'A car driving down a street under a traffic light.', 'A traffic light sitting on the side of a road.', 'A city street filled with lots of traffic.', 'A city street filled with lots of traffic.', 'A traffic light sitting on the side of a road.', 'A city street filled with lots of traffic.', 'A city street filled with lots of traffic.', 'A city street filled with lots of traffic.', 'A city street lined with parked cars and trees.', 'A white SUV driving down a street next to tall buildings.', 'A fire hydrant on the side of the road.', 'A city street filled with lots of traffic.', 'A city street filled with lots of traffic.', 'A city street filled with lots of traffic.', 'A man riding a motorcycle down a street next to tall buildings.', 'A view of a city street with a bridge in the background.', 'A city street filled with lots of traffic.', 'A city street filled with lots of traffic.', 'A street scene with trees and bushes in the background.', 'A street scene with trees and a fire hydrant.', 'A car stopped at a red light at an intersection.', 'A large truck driving down a street next to tall buildings.', 'A bus that is sitting in the street.', 'A bunch of cars that are sitting in the street.', 'A city street with traffic lights and street signs.', 'A city street filled with lots of traffic.', 'A bus driving down a street next to tall buildings.', 'A bus driving down a street next to a tall building.', 'A bunch of cars that are sitting in the street.', 'A man riding a skateboard down a street next to tall buildings.', 'A white car parked on the side of a road.', 'A park bench sitting in the middle of a lush green forest.', 'A view of a street with cars driving down it.', 'A city street filled with lots of traffic.', 'A traffic light sitting on the side of a road.', 'A bunch of cars that are sitting in the street.']
# CAPTIONS_coco_beam = ['A large semi truck driving down a street.', 'A view of a street with a fire hydrant on the side of the road.', 'A city street filled with lots of traffic.', 'A couple of red fire hydrants sitting on the side of a road.', 'A city street lined with trees and buildings.', 'A traffic light sitting on the side of a road.', 'A city street filled with lots of traffic.', 'A view of a street with a bridge in the background.', 'A silver car driving down a street next to tall buildings.', 'A red stop sign sitting on the side of a road.', 'A traffic light sitting on the side of a road.', 'A city street filled with lots of traffic.', 'A city street filled with lots of traffic.', 'A city street lined with trees and bushes.', 'A city street filled with lots of traffic.', 'A traffic light sitting on the side of a road.', 'A traffic light sitting on the side of a road.', 'A city street filled with lots of traffic.', 'A blurry image of a city street filled with traffic.', 'A city street filled with lots of traffic.', 'A view of a street in the middle of the day.', 'A city street filled with lots of traffic.', 'A city street filled with lots of traffic.', 'A traffic light sitting on the side of a road.', 'A car parked on the side of the road near a fire hydrant.', 'Cars are stopped at a red light in the middle of the road.', 'A white car driving down a street next to tall buildings.', 'A city street at night with traffic lights and street signs.', 'A traffic light sitting on the side of a road.', 'A fire hydrant on the side of the road.', 'A city street filled with lots of traffic.', 'A city street filled with lots of traffic.', 'A city street filled with lots of traffic.', 'A man riding a motorcycle down a street next to tall buildings.', 'A traffic light sitting on the side of a road.', 'A city street filled with lots of traffic.', 'A city street filled with lots of traffic.', 'A street scene with trees and bushes in the background.', 'A street scene with trees and a fire hydrant.', 'A car stopped at a red light at an intersection.', 'A car that is sitting in the street.', 'A bus that is sitting in the street.', 'A bunch of cars that are sitting in the street.']


In [ ]:
from matplotlib import gridspec

plt.figure(figsize=(5, 55),dpi=250)
for saved_idx, saved in enumerate(SAVED):
    plt.subplot(len(SAVED), 3, saved_idx+1)
    plt.imshow(rearrange(dataset[saved][0], "c h w -> h w c"))
    plt.title(CAPTIONS_coco_beam[saved_idx], fontsize=3)
    plt.axis('off')
plt.show()
#     print(CAPTIONS_conc[saved_idx])
#     print(CAPTIONS_conc_beam[saved_idx])
#     print(CAPTIONS_coco[saved_idx])
#     print(CAPTIONS_coco_beam[saved_idx])


In [ ]:
COOL_CASES = [0, 11, 13, 20, 21, 25, 28, 33, 37, 39, 40, 41, 45, 48]
POOR_CASES = [3, 4, 5, 26, 34, -5]


In [ ]:
import numpy as np

plt.figure(figsize=(6, 40),dpi=400)
for cool_idx, saved_idx in enumerate(COOL_CASES):
    saved = SAVED[saved_idx]
    img, pc = dataset[saved]

    # Plot lidar
    plt.subplot(len(COOL_CASES), 2, 2*cool_idx+1)
    plt.scatter(-pc[:,1], pc[:,0], s=0.1, c=np.clip(pc[:, 3], 0, 1), cmap="coolwarm")
    plt.axis("equal")
    plt.xlim(-20, 20)
    plt.ylim(0, 40)
    plt.title(CAPTIONS_coco_beam[saved_idx], fontsize=12)
    plt.axis('off')

    # Plot image
    plt.subplot(len(COOL_CASES), 2, 2*cool_idx+2)
    plt.imshow(rearrange(img, "c h w -> h w c"))
#     plt.title(CAPTIONS_coco_beam[saved_idx], fontsize=3)
    plt.axis("off")

plt.show()


In [ ]:
COOL_CASES = [0, 11, 13, 20, 21, 25, 28, 33, 37, 39, 40, 41, 45, 48]
import numpy as np

for cool_idx, saved_idx in enumerate(COOL_CASES):
    saved = SAVED[saved_idx]
    img, pc = dataset[saved]
    
    plt.figure(figsize=(10,5),dpi=250)
    
    # Plot lidar
    plt.suptitle(CAPTIONS_coco_beam[saved_idx], fontsize=24)
    plt.subplot(121)
    plt.scatter(-pc[:,1], pc[:,0], s=0.1, c=np.clip(pc[:, 3], 0, 1), cmap="coolwarm")
    plt.axis("equal")
    plt.xlim(-20, 20)
    plt.ylim(0, 40)
    plt.axis('off')

    # Plot image
    plt.subplot(122)
    plt.imshow(rearrange(img, "c h w -> h w c"))
    plt.axis("off")

    plt.show()



In [ ]:
import numpy as np

for saved_idx in POOR_CASES:
    saved = SAVED[saved_idx]
    img, pc = dataset[saved]
    
    plt.figure(figsize=(10,5),dpi=250)
    
    # Plot lidar
    plt.suptitle(CAPTIONS_coco_beam[saved_idx], fontsize=24)
    plt.subplot(121)
    plt.scatter(-pc[:,1], pc[:,0], s=0.1, c=np.clip(pc[:, 3], 0, 1), cmap="coolwarm")
    plt.axis("equal")
    plt.xlim(-20, 20)
    plt.ylim(0, 40)
    plt.axis('off')

    # Plot image
    plt.subplot(122)
    plt.imshow(rearrange(img, "c h w -> h w c"))
    plt.axis("off")

    plt.show()


